### Восстановление пропущенных пробелов в тексте с помощью NLP / DL / алгоритма.

In [ ]:
import re, math, ast
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from sklearn.metrics import precision_recall_fscore_support
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments, DataCollatorForTokenClassification
import ast
import numpy as np

2025-09-22 14:39:34.442773: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758551974.641780      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758551974.696003      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Обучение модели

In [ ]:
# 1. Загружаем подготовленный датасет
df = pd.read_csv("/kaggle/input/train-ds/train_dataset.csv")

In [ ]:
df.head()

,id,id_1,id_2,text_1,text_2,class,text_no_space,text_with_space,labels
0,1,201,8159,полицейским разрешат стрелять на поражение по ...,Полиции могут разрешить стрелять по хулиганам ...,0,полицейскимразрешатстрелятьнапоражениепогражда...,полицейским разрешат стрелять на поражение по ...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
1,2,202,8158,право полицейских на проникновение в жилище ре...,Правила внесудебного проникновения полицейских...,0,правополицейскихнапроникновениевжилищерешилиог...,право полицейских на проникновение в жилище ре...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,3,273,8167,президент египта ввел чрезвычайное положение в...,Власти Египта угрожают ввести в стране чрезвыч...,0,президентегиптаввелчрезвычайноеположениевмятеж...,президент египта ввел чрезвычайное положение в...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ..."
3,4,220,8160,вернувшихся из сирии россиян волнует вопрос тр...,Самолеты МЧС вывезут россиян из разрушенной Си...,-1,вернувшихсяизсириироссиянволнуетвопрострудоуст...,вернувшихся из сирии россиян волнует вопрос тр...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
4,5,223,8160,в москву из сирии вернулись 2 самолета мчс с р...,Самолеты МЧС вывезут россиян из разрушенной Си...,0,вмосквуизсириивернулись2самолетамчссроссиянами...,в москву из сирии вернулись 2 самолета мчс с р...,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, ..."


In [ ]:
tokenizer = BertTokenizerFast.from_pretrained("DeepPavlov/rubert-base-cased")

In [ ]:
def split_chars(example):
    return {"chars": list(example["text_no_space"])}

In [ ]:
df["labels"] = df["labels"].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [ ]:
hf_ds = Dataset.from_pandas(df)
hf_ds = hf_ds.map(split_chars)

Map:   0%|          | 0/7227 [00:00<?, ? examples/s]

In [ ]:
def tok_align(examples):
    # Токенизация текста на уровне символов
    # is_split_into_words=True говорит токенизатору, что вход уже разбит на "слова" (в данном случае это символы)
    # truncation=True обрезает последовательности длиннее max_length
    # padding="max_length" дополняет до фиксированной длины 256
    tokenized = tokenizer(
        examples["chars"],
        is_split_into_words=True,
        truncation=True,
        padding="max_length",
        max_length=256,
    )

    # Хранит выровненные метки
    all_labels = []
    for i, labs in enumerate(examples["labels"]):
        # Получаем для текущего примера соответствие токенов и исходных символов
        word_ids = tokenized.word_ids(batch_index=i)
        aligned = [] # выровненные метки для текущего примера
        prev_idx = None
        for word_idx in word_ids:
            if word_idx is None:
                # Токены типа [CLS], [SEP], паддинги — игнорируем, ставим -100
                aligned.append(-100)
            elif word_idx != prev_idx:
                # Если токен соответствует новому символу (новое "слово"), добавляем метку
                aligned.append(labs[word_idx])
            else:
                # Если токен — продолжение предыдущего символа (WordPiece), игнорируем
                aligned.append(-100)
            prev_idx = word_idx
        all_labels.append(aligned)

    tokenized["labels"] = all_labels
    return tokenized

tokenized_ds = hf_ds.map(tok_align, batched=True, remove_columns=hf_ds.column_names)

Map:   0%|          | 0/7227 [00:00<?, ? examples/s]

In [ ]:
# Разбиваем на тренировочный и валидационный датасет
split_ds = tokenized_ds.train_test_split(test_size=0.1, seed=42)
train_ds = split_ds["train"]
val_ds = split_ds["test"]

In [ ]:
# 3. Модель RuBERT + кастомная инициализация головы
model = BertForTokenClassification.from_pretrained("DeepPavlov/rubert-base-cased", num_labels=2)

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Подбираем веса вручную
class_weight = [1.0, 2.5] # [1.0, 2.0] [1.0, 2.25] [1.0, 3.0]

In [ ]:
# 4. Кастомный Trainer с взвешенным лоссом

class TrainerWithWeightedLoss(Trainer):
    def __init__(self, class_weight, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.class_weight = torch.tensor(class_weight, dtype=torch.float)

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.get("labels")
        outputs = model(**{k: v for k, v in inputs.items() if k != "labels"})
        logits = outputs.logits

        device = next(model.parameters()).device
        num_labels = getattr(model, "num_labels", model.module.num_labels)  # универсально

        loss_fct = torch.nn.CrossEntropyLoss(
            weight=self.class_weight.to(device),
            ignore_index=-100
        )
        loss = loss_fct(logits.view(-1, num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

In [ ]:
# 5. Метрика F1 по строкам
def compute_metrics_for_stepik(eval_pred):
    logits, label_ids = eval_pred
    preds = np.argmax(logits, axis=-1)

    f1s, precs, recs = [], [], []
    for pred_row, label_row in zip(preds, label_ids):
        valid_idx = np.where(label_row != -100)[0]
        if len(valid_idx) == 0:
            continue

        true_pos = set(np.where(label_row == 1)[0]) & set(valid_idx)
        pred_pos = set(np.where(pred_row == 1)[0]) & set(valid_idx)

        if len(true_pos) == 0 and len(pred_pos) == 0:
            f1s.append(1.0); precs.append(1.0); recs.append(1.0); continue

        tp = len(pred_pos & true_pos)
        prec = tp / len(pred_pos) if len(pred_pos) > 0 else 0.0
        rec = tp / len(true_pos) if len(true_pos) > 0 else 0.0
        f1 = (2 * prec * rec) / (prec + rec) if (prec + rec) > 0 else 0.0
        f1s.append(f1); precs.append(prec); recs.append(rec)

    return {
        "precision_macro_per_line": float(np.mean(precs)) if precs else 0.0,
        "recall_macro_per_line": float(np.mean(recs)) if recs else 0.0,
        "f1_macro_per_line": float(np.mean(f1s)) if f1s else 0.0,
    }

In [ ]:
# 6. Обучение

training_args = TrainingArguments(
    output_dir="./rubert-spacefix",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=6,
    weight_decay=0.01,
    logging_steps=200,
    load_best_model_at_end=True,
    metric_for_best_model="f1_macro_per_line",
    report_to="none"
)

trainer = TrainerWithWeightedLoss(
    class_weight=class_weight,
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_for_stepik,
)


/tmp/ipykernel_36/359190239.py:6: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `TrainerWithWeightedLoss.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision Macro Per Line,Recall Macro Per Line,F1 Macro Per Line
1,0.256600,0.290924,0.589249,0.756299,0.654129
2,0.223000,0.323665,0.638761,0.701795,0.663326
3,0.202600,0.284153,0.651102,0.798710,0.710348
4,0.174800,0.300139,0.698411,0.760146,0.723017
5,0.165100,0.288200,0.703629,0.794431,0.740877
6,0.147300,0.286458,0.712769,0.796231,0.746512


TrainOutput(global_step=2442, training_loss=0.19556625664575875, metrics={'train_runtime': 1467.0968, 'train_samples_per_second': 26.599, 'train_steps_per_second': 1.665, 'total_flos': 5098422317432832.0, 'train_loss': 0.19556625664575875, 'epoch': 6.0})

In [ ]:
# Итоговая оценка на валидации
trainer.evaluate()

{'eval_loss': 0.286458283662796,
 'eval_precision_macro_per_line': 0.7127687219126078,
 'eval_recall_macro_per_line': 0.7962312315009411,
 'eval_f1_macro_per_line': 0.7465118258650287,
 'eval_runtime': 8.8311,
 'eval_samples_per_second': 81.87,
 'eval_steps_per_second': 5.209,
 'epoch': 6.0}

In [ ]:
# Сохраняем модель
model.save_pretrained("/kaggle/working/")

# Сохраняем токенизатор
tokenizer.save_pretrained("/kaggle/working/")

('/kaggle/working/tokenizer_config.json',
 '/kaggle/working/special_tokens_map.json',
 '/kaggle/working/vocab.txt',
 '/kaggle/working/added_tokens.json',
 '/kaggle/working/tokenizer.json')

### Инференс

In [ ]:
# Загружаем файл со степика
# Читаем файл построчно
with open("/kaggle/input/test-dataset/dataset_1937770_3.txt", 'r', encoding='utf-8') as f:
    lines = f.readlines()

# Обрабатываем строки
data = []
headers = lines[0].strip().split(',')

for line in lines[1:]:
    line = line.strip()
    # Разделяем только по ПЕРВОЙ запятой
    parts = line.split(',', 1)
    if len(parts) == 2:
        data.append(parts)

# Создаем DataFrame
task_data = pd.DataFrame(data, columns=headers)
task_data['id'] = task_data['id'].astype(int)

print(task_data.head())
print(f"Размер данных: {task_data.shape}")

   id                 text_no_spaces
0   0                куплюайфон14про
1   1             ищудомвПодмосковье
2   2  сдаюквартирусмебельюитехникой
3   3     новыйдивандоставканедорого
4   4                отдамдаромкошку
Размер данных: (1005, 2)


In [ ]:
# Переводим модель в режим инференса
model.eval()

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-1

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [ ]:
# Список для позиций пробелов
predicted_positions = []

for text in task_data["text_no_spaces"]:
    # Разбиваем текст на отдельные символы
    chars = list(text)

    # Токенизация
    enc = tokenizer(
        chars,
        is_split_into_words=True,
        return_tensors="pt", # возвращает PyTorch тензоры
        truncation=True  # обрезаем, если длиннее max_length
    )

    # Переносим входы на тот же девайс, где модель
    input_ids = enc["input_ids"].to(device)
    attention_mask = enc["attention_mask"].to(device)

    # Отключаем вычисление градиентов
    with torch.no_grad():
        logits = model(input_ids=input_ids, attention_mask=attention_mask).logits

    # Берем предсказание класса для каждого токена (0 = нет пробела, 1 = пробел)
    preds = torch.argmax(logits, dim=-1).squeeze().cpu().numpy()

    # Получаем индексы слов для токенов, чтобы выровнять предсказания по символам
    word_ids = enc.word_ids()
    positions = []  # список позиций пробелов для этой строки
    prev_idx = None

    # Проходим по токенам
    for idx, word_idx in enumerate(word_ids):
        if word_idx is None:
            continue
        # Если метка = 1 и это первый токен символа в слове, сохраняем позицию
        if word_idx != prev_idx and preds[idx] == 1:
            positions.append(word_idx)
        prev_idx = word_idx

    predicted_positions.append(str(positions))

In [ ]:
task_data["predicted_positions"] = predicted_positions

# Сохраняем CSV для отправки
task_data.to_csv("submission.csv", index=False)